# Ch3 LLM, RAG 개인 도전과제

### Goal: LangSmith의 Prompt Library를 참고하여 prompt engineering을 수행

##### 1. 사용환경 준비

In [1]:
import os
from getpass import getpass

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"  # 라이브러리끼리의 충돌을 해결
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

##### 2. 모델 로드하기

In [2]:
from langchain_openai import ChatOpenAI

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

##### 3. 문서 로드하기

In [3]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("files/인공지능산업최신동향_2024년_11월호.pdf")

# 페이지 별 문서 로드
docs = loader.load()

##### 4. 문서 청크로 나누기

**RecursiveCharacterTextSplitter**
- 주어진 문자 목록의 순서대로 청크가 충분히 작아질 때까지 재귀적으로 텍스트를 분할하는 클래스
- 문자 목록을 매개변수로 받아 동작
- 기본 문자 목록: \["\n\n", "\n", " ", ""\] (default)
- 텍스트를 재귀적으로 분할하여 의미적으로 관련있는 텍스트 조각들이 같이 있도록 하는 목적으로 설계됨

---
- `chunk_size`: 각 청크의 최대 길이
- `chunk_overlap`: 인접한 청크 사이에 중복으로 포함될 문자의 수
- `length_function`: 청크의 길이를 계산하는 함수
- `is_separator_regex`: 구분자로 정규식을 사용할지 여부를 설정

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

##### 5. 벡터 임베딩 생성

In [6]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

##### 6. 벡터 스토어 생성

In [7]:
from langchain_community.vectorstores import FAISS


vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

##### 7. FAISS를 Retriever로 변환

In [8]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

##### 8. 프롬프트 템플릿 정의

In [ ]:
# LangChain library에서 prompt, model, chain, agent 등을 불러오는 라이브러리
from langchain import hub

# Prompts 디렉토리가 없다면 Prompts 디렉토리 만들기
os.makedirs("Prompts", exist_ok=True)

# LangChain library에서 prompt 가져오기
prompt1 = hub.pull("rlm/rag-prompt")
prompt2 = hub.pull("rlm/rag-document-relevance")
prompt3 = hub.pull("rlm/rag-answer-hallucination")

# 가져온 prompt들을 prompts list에 저장
prompts = [prompt1, prompt2, prompt3]

d:\Anaconda3_envs\SpartaProjects\Personal_Project\project2\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
d:\Anaconda3_envs\SpartaProjects\Personal_Project\project2\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
d:\Anaconda3_envs\SpartaProjects\Personal_Project\project2\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
# Prompts 디렉토리에 prompt 저장
for i in range(1, 4):
    # 파일 경로를 .\Prompts\prompt.txt 형태로 설정
    file_path = os.path.join(".\Prompts", f"prompt{i}.txt")
    
    # ChatPromptTemplate을 str 타입으로 변환
    prompt = prompts[i-1].format(question="question", context="context")
    
    
    with open(file_path, 'w') as f:
        f.write(prompt)

In [12]:
from langchain_core.prompts import ChatPromptTemplate

prompts = {}
for i in range(1, 4):
    file_path = os.path.join(".\Prompts", f"prompt{i}.txt")
    with open(file_path, 'r') as f:
        prompts[f"prompt{i}"] = f.read()
        
for i in range(1, 4):
    print(prompts[f"prompt{i}"], "\n\n")


Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: question 
Context: context 
Answer: 


System: You are a grader assessing relevance of a retrieved document to a user question.

If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n

It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n

Give a binary score 1 or 0 score, where 1 means that the document is relevant to the question.
Human: Retrieved documents:  

User question:  


System: You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. 

Give a binary score 1 or 0, where 1 means that the answer is grounded in / supported by the set of facts.
Human: Facts:  

LLM generation:  




In [10]:
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

##### 9. RAG 체인 구성

In [11]:
from langchain_core.runnables import RunnablePassthrough

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText() | contextual_prompt | model


##### 10. 챗봇 구동 확인

In [12]:
i=0
while i<3:
	print("========================")
	query = input("질문을 입력하세요: ")
	response = rag_chain_debug.invoke(query)
	print("Final Response:")
	print(response.content)
	i += 1

Debug Output: 
Debug Output: {'context': [Document(metadata={'source': 'files/인공지능산업최신동향_2024년_11월호.pdf', 'page': 1}, page_content='·····························21')], 'question': ''}
Final Response:
It seems that there is no specific question provided. Could you please provide more details or clarify your question?
Debug Output: 
Debug Output: {'context': [Document(metadata={'source': 'files/인공지능산업최신동향_2024년_11월호.pdf', 'page': 1}, page_content='·····························21')], 'question': ''}
Final Response:
It seems that the question is incomplete or not provided. Please provide the full question for me to assist you effectively.
Debug Output: 
Debug Output: {'context': [Document(metadata={'source': 'files/인공지능산업최신동향_2024년_11월호.pdf', 'page': 1}, page_content='·····························21')], 'question': ''}
Final Response:
It seems that the context provided is incomplete and does not contain sufficient information to address any specific question. Please provide more details or

In [13]:
# 일반 ChatGPT와 답변 비교
llm = ChatOpenAI(model="gpt-4o-mini")

for i in range(3):
    print("========================")
    query = input("질문을 입력하세요: ")
    print(f"query: {query}")
    response = llm.invoke(query)
    print("Final Response:")
    print(response.content)

query: 
Final Response:
Hello! How can I assist you today?
query: 
Final Response:
Hello! How can I assist you today?
query: 
Final Response:
Hello! How can I assist you today?


***RAG VS 일반 GPT***

1. 정보 접근
- 일반 GPT 모델
    - 사전에 학습된 데이터에 의존하여 대답
    - 사전 학습 데이터 이후의 정보 혹은 학습된 지식을 뛰어 넘는 정보에 대한 질문은 답변하기 어려움
- RAG 모델
    - 외부 검색 시스템이나 데이터베이스로부터 <span style="color:lime">필요한 정보를 검색</span>하여 응답 생성에 활용
    - 일반 GPT 모델보다 최신 정보를 제공하기에 알맞음

2. 데이터 출처의 신뢰성
- 일반 GPT 모델
    - 학습된 정보를 바탕으로 추론을 하므로, 정보의 출처를 명확히 제시할 수 없고 특정한 사실에 대해 신뢰도가 떨어질 수 있음
- RAG 모델
    - 필요한 정보를 검색하는 과정을 거치므로, 정보의 출처를 확인할 수 있고 검색된 자료의 신뢰도를 판단할 수 있음

3. 학습된 지식 업데이트
- 일반 GPT 모델
    - 모델을 다시 훈련하지 않는 이상 지식이 업데이트 되지 않음
- RAG 모델
    - 새로운 정보를 검색함으로써 지식의 업데이트 없이도 실시간 데이터에 접근 가능

4. 요약
- **최신 정보나 특정 지식을 필요로 하는 경우, 한정된 지식으로 인해 생길 수 있는 문제를 해소하고 보다 신뢰성 높은 정보를 제공하기 위해 RAG 모델이 필요**
- RAG 모델은 카카오의 신규 AI 서비스인 카나나에 대해 잘 설명한 반면, 일반 모델은 그렇지 못함